# Задачи, цели работы


## Цель работы


Цель этой лабораторной работы - изучить и применить готовые библиотеки для оптимизации на Python, в частности, PyTorch и различные методы оптимизации из SciPy. Также, требуется сравнить эффективность работы этих методов с нашими реализациями из прошлых работ.

## Задачи для достижения указанной цели

1. Изучить использование вариантов SGD (torch.optim) из PyTorch. Исследовать их эффективность и сравнить с собственными реализациями из предыдущих работ.
2. Изучить использование готовых методов оптимизации из SciPy (scipy.optimize.minimize, scipy.optimize.least_squares). Исследовать их эффективность и сравнить с собственными реализациями из предыдущих работ.
3. Реализовать использование PyTorch для вычисления градиента и сравнить его с другими подходами.
4. Исследовать, как задание границ изменения параметров влияет на работу методов из SciPy.
5. Исследовать использование линейных и нелинейных ограничений при использовании scipy.optimize.minimize из SciPy. Рассмотреть случаи, когда минимум находится на границе заданной области и когда он расположен внутри.
6. Подготовить отчет, содержащий описание реализованных алгоритмов, реализацию, необходимые тесты и таблицы. Отчет должен также включать анализ результатов, преимуществ и ограничений методов.

# Ход работы

## Подготовка среды, определение полезных функций

### В предыдущих сериях:

In [ ]:
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
import profiler
import descent
import regression
import visualization
import dataset

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)
np.set_printoptions(precision=2, suppress=False)

### Dataset

В качестве задачи Least Squares мы будем использовать модель нелинейной регрессии из прошлой работы на базе датасета изменения погоды от Яндекса:

In [ ]:
data = dataset.get_data()
data_weather = dataset.get_data_weather()
visualization.visualize_regression([0.0], *data_weather)

Как видим, данные вполне периодичны. Попробуем их описать синусом:

Определим модель как
$$M(x, W) = W_1 + W_2 \cdot x + W_3 \cdot x^2 + W_4 \cdot x^3 + W_5 \cdot \sin (x \cdot W_6 + W_7)$$
Тогда Loss-функция будет выглядеть как
$$
f(W) = \sum_{i\in{DATA_x}} (DATA_{yi} - M(i))^2
$$

In [ ]:
f_weather, f_weather_chunk = dataset.get_nonlinear_loss_func(*data_weather)

В качестве квадратичной задачи для градиентного спуска, будем использовать Loss-функцию линейной регрессии из работы №2. Она построена на базе датасета успеваемости студентов в зависимости от количества часов подготовки:

In [ ]:
f_stud, f_stud_chunk = dataset.get_linear_loss_func(*data)
visualization.visualize_regression([0.0], *data)

### Rosenbrock function

Она пригодится для сравнения работы различных алгоритмов оптимизации

Определим её следующим образом:
$$
f(x, y) = (1 - x)^2 + 100(y - x^2)^2
$$
Это нелинейная функция, будем использовать её минимизировать для демонстрации работы некоторых методов в будущем.

## Задание 1. Исследование реализации SGD из Pytorch

### Наша реализация

В прошлых работах были реализованы методы:
- Sgd
- Sgd with momentum
- Rmsprop
- Adagrad
- Adam

In [ ]:
from descent import (
    minibatch_descent,
    momentum_minibatch_descent,
    rmsprop_minibatch_descent,
    adagrad_minibatch_descent,
    adam_minibatch_descent,
)

#### Функция Розенброка

In [ ]:
from funcs import squared, f_rosenbrock, f_rosenbrock_chunk

print("Глобальный минимум: [1.0, 1.0]")
visualization.visualize_multiple_descent_2args(
    {
        "DIY Adam": adam_minibatch_descent(
            f=squared(f_rosenbrock_chunk()),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            decay=descent.step_decay(1.0, 0.8, 100),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
            betta1=0.9,
            betta2=0.9,
        ),
        "DIY SGD": minibatch_descent(
            f=squared(f_rosenbrock_chunk()),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            lr=descent.constant_lr_decay(0.00001),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
        ),
        "DIY Momentum SGD": momentum_minibatch_descent(
            f=squared(f_rosenbrock_chunk()),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            lr=descent.constant_lr_decay(0.0001),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
            alpha=0.9,
        ),
        "DIY RMSProp": rmsprop_minibatch_descent(
            f=squared(f_rosenbrock_chunk()),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            lr=descent.step_decay(1.0, 0.5, 100),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
            alpha=0.9,
        ),
        "DIY Adagrad": adagrad_minibatch_descent(
            f=squared(f_rosenbrock_chunk()),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            lr=descent.constant_lr_decay(1.0),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
        ),
    },
    f_rosenbrock,
    print_points=True,
)

Ввиду обусловленности представленной функции, обычный стохастический спуск так и не удалось адекватно стабилизировать, он начал сходиться только при чрезвычайно маленьком Learning Rate (1e-5). Каждое его улучшение помогает улучшить и ускорить сходимость, что отлично видно на графиках, учитывая подобранный Learning Rate для каждого из методов.

#### Линейная регрессия

In [ ]:
from funcs import squared

visualization.linear_multiple_demo_2args(
    {
        "DIY Adam": adam_minibatch_descent(
            f=squared(f_stud_chunk),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            decay=descent.step_decay(1.0, 0.8, 100),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
            betta1=0.9,
            betta2=0.9,
        ),
        "DIY Adagrad": adagrad_minibatch_descent(
            f=squared(f_stud_chunk),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            lr=descent.constant_lr_decay(1.0),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
        ),
        "DIY SGD": minibatch_descent(
            f=squared(f_stud_chunk),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            lr=descent.constant_lr_decay(0.00001),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
        ),
        "DIY Momentum SGD": momentum_minibatch_descent(
            f=squared(f_stud_chunk),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            lr=descent.constant_lr_decay(0.0001),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
            alpha=0.9,
        ),
        "DIY RMSProp": rmsprop_minibatch_descent(
            f=squared(f_stud_chunk),
            df=descent.numeric_gradient,
            x0=np.array([-10.0, -10.0]),
            lr=descent.step_decay(1.0, 0.5, 100),
            n_epochs=1000,
            batch_size=1,
            tol=0.01,
            alpha=0.9,
        ),
    },
    f_stud,
    *data
)

### Pytorch

Подготовим аналогичные методы в реализации PyTorch

#### Функция Розенброка

#### Линейная регрессия

### Вывод

// Вывод


## Задание 2. Исследование методов оптимизации SciPy


### Сравнение с прошлой работой

#### scipy.optimize.minimize

#### scipy.optimize.least_squares

### Вычисление градиента с помощью PyTorch, сравнение

#### scipy.optimize.minimize

#### scipy.optimize.least_squares

### Изменение параметров и границ, сравнение

#### scipy.optimize.minimize

#### scipy.optimize.least_squares

### Вывод

// Вывод

# Бонусное задание

## Задание 1. Исследование использования ограничений SciPy при работе с scipy.optimize.minimize

### scipy.optimize.LinearConstraint

// :NOTE: Рассмотреть случаи когда минимум находится на границе заданной области и когда он расположен внутри

### scipy.optimize.NonlinearConstraint

// :NOTE: Рассмотреть случаи когда минимум находится на границе заданной области и когда он расположен внутри

### Вывод

// Вывод

# Заключение



// вывод